In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
from state_cleaner import *
from datetime import datetime
import requests
import json

%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
os.chdir('../..')
os.chdir('CovidMobile')
os.getcwd()

'/Users/samismalling/Documents/mobility-report-data-extractor-master/CovidMobile'

In [3]:
fips_to_county = pd.read_csv('./data/ZIP-COUNTY-FIPS_2017-06.csv')
parks = pd.read_csv('./data/parks_only.csv')

In [4]:
fips = parks['FIPS'].unique()
fips = fips[1:]
len(fips)

955

In [5]:
fips_to_zips = dict()
for fip in fips:
    fips_to_zips[int(fip)] = list(fips_to_county[fips_to_county['STCOUNTYFP']==fip]['ZIP'])

In [6]:
def get_weather(locationid, datasetid, begin_date, end_date, mytoken, base_url):
    token = {'token': mytoken}
    
    #passing as string instead of dict because NOAA API does not like percent encoding
    params = 'datasetid='+str(datasetid)+'&'+'locationid='+str(locationid)+'&'+'startdate='+str(begin_date)+'&'+'enddate='+str(end_date)+'&'+'limit=25'+'&datatypeid=TOBS,PRCP'
    
    r = requests.get(base_url+params, headers=token)
    #print("Request status code: "+str(r.status_code))

    try:
        #results comes in json form. Convert to dataframe
        df = pd.DataFrame.from_dict(r.json()['results'])
        #print("Successfully retrieved "+str(len(df['station'].unique()))+" stations")
        dates = pd.to_datetime(df['date'])

        return df
    except:
        pass

In [7]:
def already_exists(fip):
    #Checks to see if the fip data is already in the database
    if conn.noaaweather.records.count_documents({ 'FIPS': fip }, limit = 1) != 0:
        return True
    else:
        return False

In [ ]:
import itertools
z = dict(itertools.islice(fips_to_zips.items(), 5))

In [8]:
import json
import pymongo
import time
from pprint import pprint

conn=pymongo.MongoClient()
db = conn.noaaweather
records = db.records
conn.list_database_names()

['admin', 'config', 'local', 'mydb', 'noaaweather']

In [34]:
baseurl = 'https://www.ncdc.noaa.gov/cdo-web/api/v2/data?'
mytoken = 'NVPWXeGxislWKqThiizoffNjnPtUtAcT'
databaseid = 'GHCND'
begindate = '2020-03-01'
enddate = '2020-05-01'
newdf = get_weather('ZIP:'+str(6673), databaseid,begindate,enddate,mytoken,baseurl)

Request status code: 200


In [ ]:
codes = fips_to_zips[6053]
for code in codes:
    try:
        newdf = get_weather('ZIP:'+str(code), databaseid,begindate,enddate,mytoken,baseurl)
        fip_weather = fip_weather.append(newdf)
    except:
        continue

In [ ]:
empty_fips = [9001,9003,9005,9007,9009,9011,9013,9015,22075,22095,23001,23003,23005,23007,
              23009,23011,23017,23019,23025,23031,25001,25003,25005,25009,25011,25013,25015,
              25017,25021,25023,25025,25027,26025,39133,39141,39151,39153,39155,39165,39173,
              40017,40021,40027,40031,40037,40047,40049,40089,40099,40101,40109,40113,40119,
              40121,40125,40131,40143,40151,41003,41005,41007,41009,41011,41015,41017,41019,
              41027,41029,41033,41035,41039,41041,41043,41047,41051,41053,41057,41059,41065,
              41067,41071,42001,42003,42007,42011,42013,42017,42019,42025,42027,42029,42033,
              42041,42043,42045,42049,42051,42055,42069,42071,42073,42075,42077,42079,42081,
              42089,42091,42095,42101,42107,42111,42115,42123,42125]

In [ ]:
baseurl = 'https://www.ncdc.noaa.gov/cdo-web/api/v2/data?'
mytoken = 'NVPWXeGxislWKqThiizoffNjnPtUtAcT'
databaseid = 'GHCND'
begindate = '2020-03-01'
enddate = '2020-05-01'

tic = time.perf_counter()
weather = pd.DataFrame()
for fip, codes in fips_to_zips.items():
    if not already_exists(fip) and fip not in empty_fips: #check to make sure not double counting data
        print(fip)
        fip_weather = pd.DataFrame()
        for code in codes:
            try:
                newdf = get_weather('ZIP:'+str(code), databaseid,begindate,enddate,mytoken,baseurl)
                fip_weather = fip_weather.append(newdf)
            except:
                continue
        if not fip_weather.empty:
            #average across the county
            precip = fip_weather[fip_weather['datatype']=='PRCP'][['date','value']].rename(columns={'value':'precipitation'})
            temp = fip_weather[fip_weather['datatype']=='TOBS'][['date','value']].rename(columns={'value':'temp'})
            w = pd.merge(precip, temp, how ='outer', on ='date') 
            avgs = pd.merge(w.groupby('date', as_index=False)['precipitation'].mean(),
                            w.groupby('date', as_index=False)['temp'].mean(),on='date')
            avgs['FIPS']=fip
            avgs = avgs.to_dict()

            #insert records into MongoDB
            a = [{key:value[index] for key,value in avgs.items()}
                 for index in range(max(map(len,avgs.values())))]
            records.insert_many(a)
            print(records.count())
        else:
            empty_fips.append(fip)
    
toc= time.perf_counter()
print(f"Run time was {toc - tic:0.4f} seconds")

33001
33003
33005
33007
33009
33011
33013
33015
33017
34001
34003
34005
34007
34009
34011
34013
34015
34017
34019
34021
34023
34025
34027
34029
34031
34033
34035
34037
34039
34041
36005
36071


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:33: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere


15885
36075
15929
36079
15976
36081
16017
36083
16045
36085
16071
36087
16096
36091
16129
36093
16157
36089
16195
36101
16237
36103
16288
36109
16313
36111
16340
36113
16373
36119
16426
37001
16453
37013
16478
37017
16509
37019
16542
37021
16573
37023
16615
37025
16643
37027
16664
37031
16695
37035
16739
37037
16768
37045
16804
37049
16817
37051
16842
37055
16868
37057
16910
37063
16946
37067
16985
37071
17021
37079
17053
37081
17109
37085
17138
37087
17163
37089
17196
37097
17230
37099
17255
37101
17284
37109
17325
37113
17338
37111
17363
37119
17398
37123
17449
37125
17494
37127
17534
37129
17559
37133
17590
37135
17623
37147
17655
37151
17707
37157
17746
37159
17771
37161
17806
37169
17845
37171
17881
37173
17894
37175
17919
37179
17954
37183
18000
37189
18013
37191
18038
37193
18081
37195
18117
38015
18168
38017
18211
38025
18260
38035
18299
39003
18331
39007
18356
39017
18398
39025
18438
39035
18495
39041
18535
39043
18594
39045
18640
39049
18683
39055
18713
39057
18770
39061
1881

#### Practice- only with DC zip codes:

In [ ]:
dc_zips = fips_to_zips['District of Columbia']
len(dc_zips)

In [ ]:
baseurl = 'https://www.ncdc.noaa.gov/cdo-web/api/v2/data?'
mytoken = 'NVPWXeGxislWKqThiizoffNjnPtUtAcT'
databaseid = 'GHCND'
begindate = '2020-03-01'
enddate = '2020-05-01'
dc_weather = pd.DataFrame()
for code in dc_zips[0:100]:
    print(code)
    try:
        newdf = get_weather('ZIP:'+str(code), databaseid,begindate,enddate,mytoken,baseurl)
        newdf.insert(0,'ZIP',code)
        dc_weather = dc_weather.append(newdf)
    except:
        continue

In [ ]:
precip = dc_weather[dc_weather['datatype']=='PRCP'][['ZIP','date','value']].rename(columns={'value':'precipitation'})
temp = dc_weather[dc_weather['datatype']=='TOBS'][['date','value']].rename(columns={'value':'temp'})
w = pd.merge(precip, temp, how ='outer', on ='date') 
avgs = pd.merge(w.groupby('date', as_index=False)['precipitation'].mean(),
                w.groupby('date', as_index=False)['temp'].mean(), on='date')
avgs ['County']='DC'
avgs.to_dict()

In [ ]:
ld = [{key:value[index] for key,value in a.items()}
         for index in range(max(map(len,a.values())))]
ld

#### Check whether weather from one ZIP is different across a county

In [ ]:
import scipy.stats as stats

In [ ]:
stats.f_oneway(weather['precipitation'][weather['ZIP'] == weather['ZIP'].unique()[0]],
               weather['precipitation'][weather['ZIP'] == weather['ZIP'].unique()[1]],
               weather['precipitation'][weather['ZIP'] == weather['ZIP'].unique()[2]])

In [ ]:
stats.f_oneway(weather['temp'][weather['ZIP'] == weather['ZIP'].unique()[0]].dropna(), 
             weather['temp'][weather['ZIP'] == weather['ZIP'].unique()[1]].dropna(),
             weather['temp'][weather['ZIP'] == weather['ZIP'].unique()[2]].dropna())

### Getting weather data using the noaa package from github:

In [ ]:
n = noaa.NOAA()

In [ ]:
forecast = n.get_observations(20001, 'US',start='2020-04-28',end='2020-04-30')
f = pd.DataFrame(forecast)
items = ['timestamp','textDescription','temperature','precipitationLast6Hours','relativeHumidity','cloudLayers']
f = f[items]
f.insert(0,'ZIP',zip_code) #add zip code 

In [ ]:
dc_weather = pd.DataFrame(columns = ['ZIP','timestamp','textDescription','temperature','precipitationLast6Hours','relativeHumidity','cloudLayers'])
dczips = [20001,20004, 20005, 20036, 20009, 20007, 20010, 20008, 20016]
for code in dczips:
    forecast = n.get_observations(code, 'US',start='2020-04-28',end='2020-04-30')
    for obs in forecast:
        time = obs['timestamp']
        desc = obs['textDescription']
        temp = obs['temperature']['value']
        precip = obs['precipitationLast6Hours']['value']
        hum = obs['relativeHumidity']['value']
        cloud = obs['cloudLayers'][0]['base']['value']
    
        row = [code,time,desc,temp,precip,hum,cloud]
        weather.loc[len(weather)] = row

In [ ]:
weather.to_csv('/Users/samismalling/Documents/mobility-report-data-extractor-master/CovidMobile/data/DC_weather_04_28-04_30.csv')

In [ ]:
weather = pd.DataFrame()
bad_zips = []
for zip_code in fips_to_county['ZIP'][0:200]:
    if zip_code not in bad_zips:
        print(zip_code)
        try:
            forecast = n.get_forecasts(zip_code, 'US',hourly = False)
            f = pd.DataFrame(forecast)
            f = f[['startTime','endTime','temperature','windSpeed','windDirection','shortForecast']]
            f.insert(0,'ZIP',zip_code) #add zip code 
            weather = pd.concat([weather, f], ignore_index = True)
        except:
            bad_zips.append(zip_code)
            continue